<a href="https://colab.research.google.com/github/a1rishav/ds_ml/blob/main/cnn/Transfer_Learning_flowers_resnet50.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os 
os.environ ["KAGGLE_USERNAME"] = ""
os.environ["KAGGLE_KEY"] = ""
!kaggle datasets download alxmamaev/flowers-recognition

flowers-recognition.zip: Skipping, found more recently modified local copy (use --force to force download)


In [11]:
!ls  

flowers-recognition.zip  gdrive  sample_data


In [5]:
from google.colab import drive
drive.mount('/content/gdrive')

root_path = "/content/gdrive/MyDrive/ai/tensorflow_keras/cnn/transfer_learning"

Mounted at /content/gdrive


Create a new directory 'data' to save your Train & Test data

In [26]:
!mv flowers-recognition.zip /content/gdrive/MyDrive/ai/tensorflow_keras/cnn/transfer_learning

mv: failed to access '/content/gdrive/MyDrive/ai/tensorflow_keras/cnn/transfer_learning': Transport endpoint is not connected


# Load Files

In [35]:
# !cd /content/gdrive/MyDrive/ai/tensorflow_keras/cnn/transfer_learning
!cd /content/gdrive/MyDrive/ai/tensorflow_keras/cnn/transfer_learning/
!ls gdrive/

/bin/bash: line 0: cd: /content/gdrive/MyDrive/ai/tensorflow_keras/cnn/transfer_learning/: Transport endpoint is not connected
ls: cannot access 'gdrive/': Transport endpoint is not connected


In [ ]:
!unzip /content/gdrive/MyDrive/ai/tensorflow_keras/cnn/transfer_learning/flowers-recognition.zip

In [38]:
import os
os.chdir(root_path)

In [40]:
!ls

flowers-recognition.zip  Transfer_Learning_Notebook_updated.ipynb


In [21]:
import os
import shutil
import numpy as np
import glob   
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, optimizers
from tensorflow.keras.layers import Input, Add,Dropout, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalAveragePooling2D
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.utils import  plot_model
from tensorflow.keras.applications.imagenet_utils import preprocess_input
from tensorflow.keras.initializers import glorot_uniform
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img, img_to_array
from tensorflow.keras.applications import ResNet50
from keras.applications.resnet import preprocess_input

from IPython.display import SVG
import scipy.misc
from matplotlib.pyplot import imshow

import random, shutil
%matplotlib inline


In [3]:
tf.test.gpu_device_name()

'/device:GPU:0'

In [67]:
HOME_FOLDER = '/content/gdrive/MyDrive/ai/tensorflow_keras/cnn/transfer_learning/flowers'

noOfFiles = 0
noOfDir = 0

for base, dirs, files in os.walk(HOME_FOLDER):
    print('Looking in : ',base)
    for directories in dirs:
        noOfDir += 1
    for Files in files:
        noOfFiles += 1

print('Number of files',noOfFiles)
print('Number of Directories',noOfDir)
print('Total:',(noOfDir + noOfFiles))

Looking in :  /content/gdrive/MyDrive/ai/tensorflow_keras/cnn/transfer_learning/flowers
Looking in :  /content/gdrive/MyDrive/ai/tensorflow_keras/cnn/transfer_learning/flowers/daisy
Looking in :  /content/gdrive/MyDrive/ai/tensorflow_keras/cnn/transfer_learning/flowers/dandelion
Looking in :  /content/gdrive/MyDrive/ai/tensorflow_keras/cnn/transfer_learning/flowers/rose
Looking in :  /content/gdrive/MyDrive/ai/tensorflow_keras/cnn/transfer_learning/flowers/sunflower
Looking in :  /content/gdrive/MyDrive/ai/tensorflow_keras/cnn/transfer_learning/flowers/tulip
Number of files 4317
Number of Directories 5
Total: 4322


In [6]:
base_dir = root_path

In [26]:
#create two folders (train and validation)
#train_dir = os.path.join(base_dir, 'train')
#os.mkdir(train_dir)
validation_dir = os.path.join(base_dir, 'validation')
if not os.path.exists(validation_dir):
  os.mkdir(validation_dir)
train_dir = os.path.join(base_dir, 'train')
if not os.path.exists(train_dir):
  os.mkdir(train_dir)
#Under train folder create five folders 
# (cloud, folly, rainy, shine, sunrise)
train_daisy_dir = os.path.join(train_dir, 'daisy')
if not os.path.exists(train_daisy_dir):
  os.mkdir(train_daisy_dir)
train_dandelion_dir = os.path.join(train_dir, 'dandelion')
if not os.path.exists(train_dandelion_dir):
  os.mkdir(train_dandelion_dir)
train_rose_dir = os.path.join(train_dir, 'rose')
if not os.path.exists(train_rose_dir):
  os.mkdir(train_rose_dir)
train_sunflower_dir = os.path.join(train_dir, 'sunflower')
if not os.path.exists(train_sunflower_dir):
  os.mkdir(train_sunflower_dir)
train_tulip_dir = os.path.join(train_dir, 'tulip')
if not os.path.exists(train_tulip_dir):
  os.mkdir(train_tulip_dir)
#Under validation folder create five folders 

   
# (cloud, folly, rainy, shine, sunrise)
validation_daisy_dir = os.path.join(validation_dir, 'daisy')
if not os.path.exists(validation_daisy_dir):
  os.mkdir(validation_daisy_dir)
validation_dandelion_dir = os.path.join(validation_dir, 'dandelion')
if not os.path.exists(validation_dandelion_dir):
  os.mkdir(validation_dandelion_dir)
validation_rose_dir = os.path.join(validation_dir, 'rose')
if not os.path.exists(validation_rose_dir):
  os.mkdir(validation_rose_dir)
validation_sunflower_dir = os.path.join(validation_dir, 'sunflower')
if not os.path.exists(validation_sunflower_dir):
  os.mkdir(validation_sunflower_dir)
validation_tulip_dir = os.path.join(validation_dir, 'tulip')
if not os.path.exists(validation_tulip_dir):
  os.mkdir(validation_tulip_dir)

# Split images to train_and_test

In [27]:
data_dir = os.path.join(base_dir, 'flowers')

train_dir = os.path.join(base_dir, 'train')
test_dir = os.path.join(base_dir, 'validation')
train_test_ratio = 0.8

def split_dataset_into_test_and_train_sets(data_dir=data_dir, train_dir = train_dir, test_dir =test_dir, train_test_ratio=train_test_ratio):
  for sub_dirs, dirs, files in os.walk(data_dir):
    if not files:
      continue
    print(sub_dirs)
    print(len(files))

    category = os.path.basename(sub_dirs)
    random.shuffle(files)
    
    train_files = files[:round(len(files)*train_test_ratio)]
    test_files = files[-round(len(files)*(1-train_test_ratio)):]

    train_category_path = os.path.join(train_dir, category)
    test_category_path = os.path.join(test_dir, category)

    for file_name in train_files:
      source_file_path = os.path.join(data_dir, category, file_name)
      shutil.copy(source_file_path, train_category_path)

    for file_name in test_files:
      source_file_path = os.path.join(data_dir, category, file_name)
      shutil.copy(source_file_path, test_category_path)

    print(len(os.listdir(train_category_path)))
    print(len(os.listdir(test_category_path)))      

    print(category)

split_dataset_into_test_and_train_sets()

/content/gdrive/MyDrive/ai/tensorflow_keras/cnn/transfer_learning/flowers/daisy
764
611
153
daisy
/content/gdrive/MyDrive/ai/tensorflow_keras/cnn/transfer_learning/flowers/dandelion
1052
842
210
dandelion
/content/gdrive/MyDrive/ai/tensorflow_keras/cnn/transfer_learning/flowers/rose
784
627
157
rose
/content/gdrive/MyDrive/ai/tensorflow_keras/cnn/transfer_learning/flowers/sunflower
733
586
147
sunflower
/content/gdrive/MyDrive/ai/tensorflow_keras/cnn/transfer_learning/flowers/tulip
984
787
197
tulip


# Use a pretrained resnet 50 as is, just replace last layers add custom softmax

In [28]:
# Number of classes in dataset
num_classes = 5

def get_model():
    # Get base model 
    # Here we are using ResNet50 as base model | Actual softmax layer we do not have to include.
    base_model = ResNet50(weights='imagenet', include_top=False)
    
    # As we are using ResNet model only for feature extraction and not adjusting the weights
    # we freeze the layers in base model
    for layer in base_model.layers:
        layer.trainable = False
        
    # Get base model output 
    base_model_ouput = base_model.output
    
    # Adding our own layer 
    x = GlobalAveragePooling2D()(base_model_ouput)
    # Adding fully connected layer
    x = Dense(512, activation='relu')(x)
    x = Dense(num_classes, activation='softmax', name='fcnew')(x)
    
    model = Model(inputs=base_model.input, outputs=x)
    return model

In [29]:
# Get the model
model = get_model()
# Compile it
model.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])
# Summary of model
model.summary()

94781440/94765736 [==============================] - 0s 0us/step
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None, None,  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, None, None, 3 0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, None, None, 6 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, None, None, 6 256         conv1_conv[0][0]                 
_____________________________

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [31]:
# Defining the imagedatagenerator for train and test image for pre-processing
# We don't give horizonal_flip or other preprocessing for validation data generator

image_size = 224
batch_size = 64

train_data_gen = ImageDataGenerator(preprocessing_function = preprocess_input,
    shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
valid_data_gen = ImageDataGenerator(preprocessing_function = preprocess_input)
train_generator = train_data_gen.flow_from_directory(train_dir, (image_size,image_size), batch_size=batch_size, class_mode='categorical')
valid_generator = valid_data_gen.flow_from_directory(test_dir, (image_size,image_size), batch_size=batch_size, class_mode='categorical')

Found 3453 images belonging to 5 classes.
Found 864 images belonging to 5 classes.


In [32]:
# Training the fully conncected layer for initial epochs
epochs = 5

# Training the model

model.fit(
    train_generator,
    steps_per_epoch=train_generator.n//batch_size,
    validation_data=valid_generator,
    validation_steps=valid_generator.n//batch_size,
    epochs=epochs)

Epoch 1/5
53/53 [==============================] - 83s 944ms/step - loss: 0.7327 - accuracy: 0.7433 - val_loss: 0.4572 - val_accuracy: 0.8329
Epoch 2/5
53/53 [==============================] - 49s 918ms/step - loss: 0.4218 - accuracy: 0.8522 - val_loss: 0.3750 - val_accuracy: 0.8714
Epoch 3/5
53/53 [==============================] - 49s 920ms/step - loss: 0.3458 - accuracy: 0.8870 - val_loss: 0.3576 - val_accuracy: 0.8606
Epoch 4/5
53/53 [==============================] - 49s 918ms/step - loss: 0.3129 - accuracy: 0.8905 - val_loss: 0.3178 - val_accuracy: 0.8930
Epoch 5/5
53/53 [==============================] - 49s 917ms/step - loss: 0.2800 - accuracy: 0.9079 - val_loss: 0.3129 - val_accuracy: 0.8906


In [ ]:
#Getting 90% & 89% accuracy after FREEZING LAYERS

# Freeze top 140 layers, retrain rest (better performance)

In [33]:
# More fine tuning the model
# Training the model after 140 layers
# Generally ResNet is good at extracting lower level features so we are not fine tuning initial layers
epochs = 10

split_at = 140                             #Freezing 140 layers and training the remaining 10 layers
for layer in model.layers[:split_at]: 
  layer.trainable = False
for layer in model.layers[split_at:]: 
  layer.trainable = True                  #Layer Training after 140 is True
    
model.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=['accuracy'])

# Choosing lower learning rate for fine-tuning
# learning rate is generally 10-1000 times lower than normal learning rate, if we are fine tuning the initial layers
sgd = optimizers.SGD(lr=0.001, decay=1e-6, momentum=0.9, nesterov=True)

model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

model.fit_generator(
    train_generator,
    steps_per_epoch=train_generator.n//batch_size,
    validation_data=valid_generator,
    validation_steps=valid_generator.n//batch_size,
    epochs=epochs,
    verbose=1)

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")
/usr/local/lib/python3.7/dist-packages/keras/engine/training.py:1972: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/10
53/53 [==============================] - 56s 967ms/step - loss: 0.3576 - accuracy: 0.8883 - val_loss: 0.3197 - val_accuracy: 0.8834
Epoch 2/10
53/53 [==============================] - 51s 962ms/step - loss: 0.2316 - accuracy: 0.9307 - val_loss: 0.2779 - val_accuracy: 0.8954
Epoch 3/10
53/53 [==============================] - 51s 958ms/step - loss: 0.1784 - accuracy: 0.9451 - val_loss: 0.2859 - val_accuracy: 0.8978
Epoch 4/10
53/53 [==============================] - 51s 954ms/step - loss: 0.1317 - accuracy: 0.9625 - val_loss: 0.2698 - val_accuracy: 0.9026
Epoch 5/10
53/53 [==============================] - 51s 958ms/step - loss: 0.1152 - accuracy: 0.9672 - val_loss: 0.2592 - val_accuracy: 0.9195
Epoch 6/10
53/53 [==============================] - 50s 950ms/step - loss: 0.0893 - accuracy: 0.9770 - val_loss: 0.2518 - val_accuracy: 0.9171
Epoch 7/10
53/53 [==============================] - 50s 948ms/step - loss: 0.0777 - accuracy: 0.9823 - val_loss: 0.2481 - val_accuracy: 0.9183

#Accuracy improved from 90%train & 89%test to 99%train & 92%test

So the last 10 layers were trained/optimized to get classification for VRC challenge of ResNet. So removing them and customizing/training them for our problem, increased the accuracy for us.

In [34]:
print('Training complete')

Training complete
